## 2. Sample_decode_job need to have attribute of num_Erasure and num_DEPOLARIZE2


### 1. how do erasure error mechanism behave when it's set to be erroneous?

Note that, it is WRONG to simply set $p_e = 1$ and inject these four events with probability $1/4$:

    II double herald, IZ double herald, ZI double herald, ZZ double herald

because most erasures exist in the form of single erasure, not double erasure

Therefore, every single erasure error mechanism should take in two bit of information during importance sampling


### 2. how do Depolarize2 error mechanism behave when it's set to be erroneous?

the original DEPOLARIZE2 introduces $24/15p_p = 8/5p_p$ Pauli errors, and if we view Y error as one X and one Z, that's $32/15p_p$ edges to be flipped in the matching graph.

But this detail is irrelevant. if we roll a dice we selected a DEPOLARIZE2 to be triggered, we just set $p_p$ = 1.

### 3. how to compute the weight of the logical error rate associated with samples of (num_Erasure = e, num_DEPOLARIZE2 = p)?

$A_{e,p}(p_e, p_p) = \binom{n_e}{e} p_e^e (1 - p_e)^{n_e - e} \times \binom{n_p}{p} p_p^t (1 - p_p)^{n_p - p}$

where $n_e$, $n_p$ is the total number of potential erasure/DEPOLARIZE2 error locations. 

For the circuit concerned, we have d noisy rounds, so there's $4*((d-1)^2+d)$ 2-qubit gates per round, and a total of $4d*((d-1)^2+d)$ 2-qubit gates

### 4. How to actually do the sampling? 

I need some sort of np.array to encode those potential error locations and then randomlly choose a certain number of them to flip

first, I need to create that numpy array


# Step-1 do a rough test that the new instruction generators are working

In [1]:
import sys
sys.path.append('..')
from surface_erasure_decoding import *
from IPython.display import clear_output


In [5]:
d = 5
after_cz_error_model = get_2q_error_model(p_e=0.03,
                                            p_z_shift=0, 
                                            p_p=0.002)
builder = easure_circ_builder(rounds = d,
                                distance= d,
                                after_cz_error_model=after_cz_error_model,
                                measurement_error=0
                                )
builder.generate_circuit_and_decoding_info()
builder.gen_erasure_conversion_circuit()

In [6]:
shots = 100

sampler = builder.erasure_circuit.compile_sampler()#expensive step, 16s for d13, 4s for d11, 0.7s for d9
meas_samples = sampler.sample(shots=shots)
converter = builder.erasure_circuit.compile_m2d_converter() # Expensive step

det_samples, actual_obs_chunk = converter.convert(measurements=meas_samples,
                                                        separate_observables=True)
new_circ_num_errors = 0
for i in range(shots):
    predicted  = builder.decode_by_generate_new_circ(det_samples[i],'S',meas_samples[i])
    new_circ_num_errors += actual_obs_chunk[i][0] != predicted
    clear_output(wait=True)
    print(f'decoding finished {100*i/shots}%')


decoding finished 99.0%


In [8]:
new_circ_num_errors

0

In [4]:
t0 = time.time()
sampler = builder.erasure_circuit.compile_sampler()#expensive step, 16s for d13, 4s for d11, 0.7s for d9

t1 = time.time()
meas_samples = sampler.sample(shots=shots)
t15 = time.time()
converter = builder.erasure_circuit.compile_m2d_converter() # Expensive step

t2 = time.time()
det_samples, actual_obs_chunk = converter.convert(measurements=meas_samples,
                                                        separate_observables=True)
t3 = time.time()
# Decode
new_circ_num_errors = 0
# normal_circ_num_errors = 0
for i in range(shots):
    predicted  = builder.decode_by_generate_new_circ(det_samples[i],'S',meas_samples[i])
    new_circ_num_errors += actual_obs_chunk[i][0] != predicted

    # predicted = builder.decode_without_changing_weights(det_samples[i],'S',meas_samples[i])
    # normal_circ_num_errors += actual_obs_chunk[i][0] != predicted

    clear_output(wait=True)
    print(f'decoding finished {100*i/shots}%')

t4 = time.time()

AttributeError: 'easure_circ_builder' object has no attribute 'erasure_circuit'

In [7]:
new_circ_num_errors

4

# generating circuit for importance sampling

### Step-1 Count how many times a certain error mechanism is triggered by calling builder.gen_dummy_circuit()

then I can access the number of times by doing mechanism.dummy_generator.num_qubit_called
```
if isinstance(attr_value, GateErrorModel):
    if not  attr_value.trivial:
        print(attr_name)
        for mechanism in attr_value.list_of_mechanisms:
            print(mechanism.name)
            print(mechanism.dummy_generator.num_qubit_called)
```

For every mechanism in non-trivial GateErrorModel(s), I can generate an array. 

In [4]:
import sys
sys.path.append('..')
from surface_erasure_decoding import *
from IPython.display import clear_output


def generate_bool_array(tot, choose):
    array = np.zeros(tot, dtype=bool)
    indices = np.random.choice(tot, choose, replace=False)
    assert len(indices) == choose
    array[indices] = True
    return array

shots = 3
d = 3

num_e_flipped = 20
num_p_flipped = 7

after_cz_error_model = get_2q_error_model(p_e=0.08,
                                            p_p=0.0033)
builder = easure_circ_builder(rounds = d,
                                distance= d,
                                after_cz_error_model=after_cz_error_model,
                                measurement_error=0
                                )
builder.generate_helper()
builder.gen_dummy_circuit()

# Step-1 get tot_e, tot_p
non_trivial_gate_error_models = [attr_value for attr_name, attr_value in vars(builder).items() if isinstance(attr_value, GateErrorModel) and not  attr_value.trivial]
assert len(non_trivial_gate_error_models) == 1

tot_e = non_trivial_gate_error_models[0].name_to_mechanism['2q erasure'].dummy_generator.num_qubit_called
tot_p = non_trivial_gate_error_models[0].name_to_mechanism['2q depo'].dummy_generator.num_qubit_called

num_qubit_per_dice_e = non_trivial_gate_error_models[0].name_to_mechanism['2q erasure'].deterministic_generator.num_qubit_per_dice
num_qubit_per_dice_p = non_trivial_gate_error_models[0].name_to_mechanism['2q depo'].deterministic_generator.num_qubit_per_dice

num_dice_e = int(tot_e/num_qubit_per_dice_e)
num_dice_p = int(tot_p/num_qubit_per_dice_p)

builder.gen_erasure_conversion_circuit()
converter = builder.erasure_circuit.compile_m2d_converter()

num_errors = 0

for i in range(shots):

    # contrary to generating erasure conversion circuit where I sync a single measurement array with multiple GateErrorModels, 
    #   here I manually set the dice for each unique mechanism
    e_dice_sample = generate_bool_array(num_dice_e, num_e_flipped)
    p_dice_sample = generate_bool_array(num_dice_p, num_p_flipped)

    non_trivial_gate_error_models[0].name_to_mechanism['2q erasure'].next_dice_index_in_list = [0]
    non_trivial_gate_error_models[0].name_to_mechanism['2q depo'].next_dice_index_in_list = [0]

    non_trivial_gate_error_models[0].name_to_mechanism['2q erasure'].single_dice_sample = e_dice_sample
    non_trivial_gate_error_models[0].name_to_mechanism['2q depo'].single_dice_sample = p_dice_sample

    builder.deterministic_circuit = stim.Circuit()
    builder.gen_circuit(builder.deterministic_circuit,mode='deterministic')

    sampler = builder.deterministic_circuit.compile_sampler()
    meas_samples = sampler.sample(shots=1)
    det_samples, actual_obs_chunk = converter.convert(measurements=meas_samples,
                                                            separate_observables=True)
    
    predicted  = builder.decode_by_generate_new_circ(det_samples[0],'S',meas_samples[0])
    num_errors += actual_obs_chunk[0][0] != predicted
    
    clear_output(wait=True)
    print(f'decoding finished {100*i/shots}%')


TypeError: 'int' object is not subscriptable